In [1]:
import pandas as pd
import datetime as dt
import dash
from dash import html,dcc
from dash.dependencies import Input,Output,State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [2]:
#I need to first look at the structure of the data
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')
#view the first 5 rows
df.head()

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced
0,NSW,1/4/2005,8.68000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R
1,NSW,1/5/2005,16.61125,322.475000,62.362500,85.500000,8.088793,65.428571,8,R
2,NSW,1/6/2005,5.52000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R
3,NSW,1/7/2005,6.26400,313.870000,33.800000,92.200000,7.529940,56.700000,5,R
4,NSW,1/8/2005,5.40000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R


In [3]:
#Statistical display of the dsts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26406 entries, 0 to 26405
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Region                               26406 non-null  object 
 1   Date                                 26406 non-null  object 
 2   Estimated_fire_area                  26406 non-null  float64
 3   Mean_estimated_fire_brightness       26406 non-null  float64
 4   Mean_estimated_fire_radiative_power  26406 non-null  float64
 5   Mean_confidence                      26406 non-null  float64
 6   Std_confidence                       24199 non-null  float64
 7   Var_confidence                       24199 non-null  float64
 8   Count                                26406 non-null  int64  
 9   Replaced                             26406 non-null  object 
dtypes: float64(6), int64(1), object(3)
memory usage: 2.0+ MB


In [4]:
#Also check more info about the data
df.describe()

,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count
count,26406.000000,26406.000000,26406.000000,26406.000000,24199.000000,24199.000000,26406.000000
mean,144.297966,319.662078,83.621258,87.574735,7.228302,56.229092,72.059305
std,314.453847,8.862005,67.510022,4.371972,1.995221,25.898935,150.973128
min,1.000000,290.700000,0.000000,76.000000,0.000000,0.000000,1.000000
25%,8.911875,313.933333,44.150391,85.000000,6.687010,44.716106,5.000000
50%,38.434091,319.784412,67.133333,87.771429,7.707025,59.398234,20.000000
75%,146.951278,325.403144,103.123611,90.498403,8.236665,67.842642,74.000000
max,10120.943170,381.950000,2178.600000,100.000000,16.970563,288.000000,3954.000000


In [5]:
#initialise dash app
app = dash.Dash(__name__)


#Extract year and month from the data
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()
df['Year'] = pd.to_datetime(df['Date']).dt.year

#apply the Layout of the App

app.layout = html.Div(children=[html.H1('Australia World Fires DashBoard',
                                        style={'textAlign':'center','color':'#503D36','font-size':26}),
                                        html.Div([html.H2('Select Region:', style={'margin-right': '2em'}),
                                                  #Radio items to select the region
                                                  dcc.RadioItems(['NSW','QL','SA','TA','VI','WA'], 'NSW', id='region',inline=True)]),
                                                  html.Div([
                                                      html.H2('Select Year:', style={'margin-right':'2em'}),
                                                      dcc.Dropdown(df.Year.unique(),value=2005,id='year')
                                                  ]),
                                                  html.Div([
                                                      html.Div([ ], id='plot1'),
                                                      html.Div([ ], id='plot2')
                                                  ],)
                                                  ])
@app.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children')],
               [Input(component_id='region', component_property='value'),
                Input(component_id='year', component_property='value')])
def reg_year_display(input_region,input_year):
    #data
    region_data = df[df['Region'] == input_region]
    y_r_data = region_data[region_data['Year'] == input_year]
    #Plot one - Monthly Average Estimated fire Area

    est_data = y_r_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()

    fig1 = px.pie(est_data, values='Estimated_fire_area', names='Month', title="{}: Monthly Average Estimated Fire Area IN YEAR {}".format(input_region,input_year))
    #Plot two - Monthly Average Count Of Pixels for Presumed Vegetation Fires
    veg_data = y_r_data.groupby('Month')['Count'].mean().reset_index()
    
    fig2 = px.bar(veg_data,x='Month', y='Count',title='{}: Average Count of Pixels for Presumed Vegetation fires in year {}'.format(input_region,input_year))
    
    return [dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2)]

if __name__ == '__main__':
    app.run_server()
